<a href="https://colab.research.google.com/github/momomendoza2476-max/SIMULACI-N-II/blob/main/simulacion_linea_de_espera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy


In [12]:
import simpy
import random

In [13]:
# parametros del sistema
random.seed(42)

tiempo_simulacion = 480        # minutos (8 horas)
media_llegadas = 5             # tiempo promedio entre llamadas
tiempo_servicio_min = 4
tiempo_servicio_mod = 6
tiempo_servicio_max = 10

In [14]:
# variables globales para indicadores
tiempo_total_cola = 0.0
llamadas_atendidas = 0

area_cola = 0.0
ultimo_evento_cola = 0.0

In [15]:
# proceso de una llamada
def llamada(env, nombre, ejecutivos):
    global tiempo_total_cola, llamadas_atendidas
    global area_cola, ultimo_evento_cola

    # tiempo en que llega la llamada
    llegada = env.now

    # actualizar area de la cola
    area_cola += len(ejecutivos.queue) * (env.now - ultimo_evento_cola)
    ultimo_evento_cola = env.now

    # la llamada solicita un ejecutivo
    with ejecutivos.request() as req:
        yield req

        # momento en que inicia el servicio
        inicio_servicio = env.now

        # acumular tiempo en cola
        tiempo_total_cola += inicio_servicio - llegada
        llamadas_atendidas += 1

        # tiempo de servicio (triangular)
        tiempo_servicio = random.triangular(
            tiempo_servicio_min,
            tiempo_servicio_max,
            tiempo_servicio_mod
        )

        yield env.timeout(tiempo_servicio)

    # actualizar area de la cola cuando sale
    area_cola += len(ejecutivos.queue) * (env.now - ultimo_evento_cola)
    ultimo_evento_cola = env.now


In [16]:
# generador de llegadas
def generador_llamadas(env, ejecutivos):
    i = 0
    while True:
        i += 1
        yield env.timeout(random.expovariate(1 / media_llegadas))
        env.process(llamada(env, f"llamada_{i}", ejecutivos))

In [17]:
# simulacion principal
env = simpy.Environment()
ejecutivos = simpy.Resource(env, capacity=num_ejecutivos)

env.process(generador_llamadas(env, ejecutivos))
env.run(until=tiempo_simulacion)


# calculo de indicadores
if llamadas_atendidas > 0:
    tiempo_promedio_cola = tiempo_total_cola / llamadas_atendidas
else:
    tiempo_promedio_cola = 0

longitud_promedio_cola = area_cola / tiempo_simulacion

utilizacion_ejecutivos = tiempo_total_cola / (tiempo_simulacion * num_ejecutivos)

In [18]:
# resultados
print("resultados de la simulacion")
print("----------------------------")
print(f"llamadas atendidas: {llamadas_atendidas}")
print(f"tiempo promedio en cola: {tiempo_promedio_cola:.2f} minutos")
print(f"longitud promedio de la cola: {longitud_promedio_cola:.2f}")
print(f"utilizacion promedio de ejecutivos: {utilizacion_ejecutivos*100:.2f} %")

resultados de la simulacion
----------------------------
llamadas atendidas: 108
tiempo promedio en cola: 0.26 minutos
longitud promedio de la cola: 0.06
utilizacion promedio de ejecutivos: 1.92 %
